# LAB 3 - LOGISTIC REGRESSION

This lab is comprised of two main sections:

- 1. Logistic Regression with only numerical variables

- 2. Logistic Regression with numerical + categorical variables

In this lab we will build up knowledge of `statsmodels`. Additionally, we will introduce another package, `scikit-learn` or `sklearn`, which is wildly used in machine learning and predictive analytics. The documentation for `sklearn` can be found at https://scikit-learn.org/stable/.

These are two of the most complete and well-documented libraries for statistical modeling in Python.

In [ ]:
import numpy as np
import pandas as pd

As usual, we summon `numpy` and `pandas` for dataset representation and manipulation.

## 1. LOGISTIC REGRESSION (ONLY NUMERICAL VARIABLES)

### 1.1 Data loading

In [ ]:
loans = pd.read_csv("loans.csv")
loans.info()
loans.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9516 entries, 0 to 9515
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   not.fully.paid  9516 non-null   int64  
 1   installment     9516 non-null   float64
 2   log.annual.inc  9516 non-null   float64
 3   fico            9516 non-null   int64  
 4   revol.bal       9516 non-null   float64
 5   inq.last.6mths  9516 non-null   int64  
 6   pub.rec         9516 non-null   int64  
dtypes: float64(3), int64(4)
memory usage: 520.5 KB


,not.fully.paid,installment,log.annual.inc,fico,revol.bal,inq.last.6mths,pub.rec
0,0,829.10,4.929419,737,28.854,0,0
1,0,228.22,4.812913,707,33.623,0,0
2,0,366.86,4.505150,682,3.511,1,0
3,0,162.34,4.929419,712,33.667,1,0
4,0,102.92,4.907411,667,4.740,0,0


### 1.2 Data cleaning and transformation

In Python, a convention is to name variables with underscores. This is slightly different from R.

Let us practice how to rename columns in `Pandas`.

In [ ]:
new_column_names = {'not.fully.paid':'not_fully_paid', 'log.annual.inc':'log_annual_inc',
                   'revol.bal':'revol_bal', 'inq.last.6mths':'inq_last_6mths', 'pub.rec':'pub_rec'}
loans.rename(columns = new_column_names, inplace = True)

print(loans.columns)

Index(['not_fully_paid', 'installment', 'log_annual_inc', 'fico', 'revol_bal',
       'inq_last_6mths', 'pub_rec'],
      dtype='object')


Use `df.describe()`, you can have a quick overview of the data set. Here, observe that the first four variables ('installment', 'log_annual_inc', 'fico', and 'revol_bal') takes continuous numeric values and the last two variables ('inq_last_6mths', 'pub_rec') takes ingeter numeric values.

In [ ]:
loans.describe()

,not_fully_paid,installment,log_annual_inc,fico,revol_bal,inq_last_6mths,pub_rec
count,9516.000000,9516.000000,9516.000000,9516.000000,9516.000000,9516.000000,9516.000000
mean,0.159836,320.131185,4.748642,710.841950,16.988484,1.572930,0.062211
std,0.366473,207.069870,0.265002,37.956246,33.721379,2.200329,0.262406
min,0.000000,15.670000,3.277838,612.000000,0.000000,0.000000,0.000000
25%,0.000000,164.020000,4.588821,682.000000,3.272750,0.000000,0.000000
50%,0.000000,269.545000,4.748188,707.000000,8.687500,1.000000,0.000000
75%,0.000000,435.405000,4.903323,737.000000,18.354250,2.000000,0.000000
max,1.000000,940.140000,6.309584,827.000000,1207.359000,33.000000,5.000000


### 1.3 Train-test split

In previous labs we saw how to split the dataset according to conditions predicated on variable's values. We now wish to split the dataset using **randomized** methods.

In order to perform the splitting, we import a package from `sklearn`. We also set a fixed random state in order to exactly replicate the results at each execution of the code.

In [ ]:
from sklearn.model_selection import train_test_split

loans_train, loans_test = train_test_split(loans, test_size=0.3, random_state=88)
#random_state: Controls the shuffling applied to the data before applying the split.
#              Pass an int for reproducible output across multiple function calls.


loans_train.shape, loans_test.shape

((6661, 7), (2855, 7))

In [ ]:
loans_train.head()

,not_fully_paid,installment,log_annual_inc,fico,revol_bal,inq_last_6mths,pub_rec
6330,0,682.74,5.079181,697,72.682,1,0
3799,0,479.21,4.778151,677,14.180,1,0
4019,0,333.15,4.579784,702,5.520,1,0
7475,0,373.32,4.819544,782,6.449,2,0
8641,0,81.51,4.417638,662,0.558,4,0


`test_size = 0.3` means that we will put 30% of the data in the test set, 70% in the training set.

### 1.4 Baseline model

In [ ]:
# How many loans have defaulted in the training set?

default_false = np.sum(loans_train['not_fully_paid'] == 0)  # not default
default_true = np.sum(loans_train['not_fully_paid'] == 1)   # default

print(pd.Series({'0': default_false, '1': default_true}))

0    5585
1    1076
dtype: int64


A baseline model can be a so-called "dummy" model, where the classifier predicts every new observation as the majority class. In our case, for a datapoint with any given features, the baseline model will always predict 'no default'.

In [ ]:
# Accuracy of baseline model based on training data:
ACC = default_false/(default_false + default_true)
ACC

0.8384626932892959

Note: we want to create models that performs better than the baseline model.

### Exercise 1: compute accuracy of baseline on testing:

In [ ]:
# # EXERCISE1: Compute accuracy of baseline on testing:
# Baseline model is constructed using training set not test set.
# test_false =  # not default
# test_true =  # default
default_false_test = np.sum(loans_test['not_fully_paid'] == 0)
default_true_test = np.sum(loans_test['not_fully_paid'] == 1)
ACC_test = default_false_test / (default_false_test + default_true_test)
ACC_test

0.8441330998248686

### Exercise2: What are the TPR and FPR rates of the baseline model?

In [ ]:
# # EXERCISE2: What are the TPR and FPR rates of the baseline model?

# # True positive: the proportion of actual positives that are correctly identified as positive
# # False positive: the proportion of actual negatives that are incorrectly identified as positive

TPR = 0                        # TPR = TP/P = TP/(TP+FN)
FPR = 0                        # FPR = FP/N = FP/(FP+TN)
print(TPR,FPR)

0 0


### Exercise3: What are the TNR and FNR rates of the baseline model?

In [ ]:
# # EXERCISE3: What are the TNR and FNR rates of the baseline model?

# # True negative: the proportion of actual negatives that are correctly identified as negative
# # False negative: the proportion of actual positives that are incorrectly identified as negative

TNR = 1                       # TNR = TN/N = TN/(TN+FP)
FNR = 1                       # FNR = FN/P = FN/(TP+FN)
print(TNR,FNR)

1 1


This is the cutoff line for lab 3. Everything beyond this line is not in scope, yet.

### 1.5 Model Fitting (Logistic Regression)

Now we can use the statsmodels package to fit the training set to a logistic regression model

In [ ]:
import statsmodels.formula.api as smf

?smf.logit

In [ ]:
# Fit the logistic regression model

logreg = smf.logit(formula = 'not_fully_paid ~ installment + log_annual_inc + fico + revol_bal + inq_last_6mths + pub_rec',
                   data = loans_train).fit()

print(logreg.summary())

Optimization terminated successfully.
         Current function value: 0.422149
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:         not_fully_paid   No. Observations:                 6661
Model:                          Logit   Df Residuals:                     6654
Method:                           MLE   Df Model:                            6
Date:                Tue, 05 Sep 2023   Pseudo R-squ.:                 0.04537
Time:                        23:33:34   Log-Likelihood:                -2811.9
converged:                       True   LL-Null:                       -2945.6
Covariance Type:            nonrobust   LLR p-value:                 8.373e-55
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          8.5765      0.967      8.866      0.000       6.681      10.472
installment      

### 1.6 Predictions

In [ ]:
loans.head()

,not_fully_paid,installment,log_annual_inc,fico,revol_bal,inq_last_6mths,pub_rec
0,0,829.10,4.929419,737,28.854,0,0
1,0,228.22,4.812913,707,33.623,0,0
2,0,366.86,4.505150,682,3.511,1,0
3,0,162.34,4.929419,712,33.667,1,0
4,0,102.92,4.907411,667,4.740,0,0


In [ ]:
# Example of prediction for a new observation

new_obs = pd.DataFrame(data = {'installment' : [366], 'log_annual_inc' : [4.51], 'fico' : [682],
                               'revol_bal' : [7.53], 'inq_last_6mths' : [1], 'pub_rec' : [0]})

logreg.predict(new_obs)  # probability of default (not paying the loan)

0    0.209408
dtype: float64

In [ ]:
y_test = loans_test['not_fully_paid']

y_prob = logreg.predict(loans_test)
y_pred = pd.Series([1 if x > 0.5 else 0 for x in y_prob], index=y_prob.index)

# y_pred is the vector of probabilities as given by your model on the test set.
# Values between 0 and 1.

# Remember, P(Yi = 1) = 1/(1 + e^(-(b0 + b1*x1 + b2*x2 +...)) )
y_pred

3007    0
4497    0
8575    0
5035    0
9037    0
       ..
7025    0
5379    0
3972    0
2827    0
7784    0
Length: 2855, dtype: int64

### 1.7 Model Evaluation - Confusion Matrix

In order to evaluate the performance of our classification model, we can make use of confusion matrix to compute a variety of useful metrics

In [ ]:
from sklearn.metrics import confusion_matrix

?confusion_matrix

To remind you of what each element of the confusion matrix represents:

TN FP

FN TP

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print ("Confusion Matrix : \n", cm)

Confusion Matrix : 
 [[2404    6]
 [ 435   10]]


In [ ]:
# compare the confusion matrix of the baseline model
confusion_matrix(loans_test['not_fully_paid'],[0]*loans_test.shape[0])

array([[2410,    0],
       [ 445,    0]])

In [ ]:
print(cm.ravel())

[2404    6  435   10]


In [ ]:
# Accuracy
(cm.ravel()[0]+cm.ravel()[3])/sum(cm.ravel())  # T/total = (TP+TN)/total

0.8455341506129597

Be careful about the definitions of FPR, TPR, recall, precision, sensitivity, specificity etc:
https://en.wikipedia.org/wiki/Sensitivity_and_specificity

### Exercise 4: Calculate TPR and FPR

In [ ]:
# EXERCISE: What is the True Positive Rate ?
TPR_logit =  cm.ravel()[3] / (cm.ravel()[3] + cm.ravel()[2] )                               #TPR = TP / P = TP / (TP + FN)
print('TPR is: %.4f' % TPR_logit)


# EXERCISE: What is the False Positive rate ?
FPR_logit = cm.ravel()[1] / (cm.ravel()[1] + cm.ravel()[0] )                         #FPR = FP / N = FP / (FP + TN)
print('FPR is: %.4f' % FPR_logit)

TPR is: 0.0225
FPR is: 0.0025


### Exercise 5: Practice with threshold prob = 0.4

In [ ]:
# Now, try threshold probability = 0.4
y_pred = pd.Series([1 if x > 0.4 else 0 for x in y_prob], index=y_prob.index)
cm = confusion_matrix(y_test,y_pred)
# EXERCISE: What is the Accuracy?
acc= (cm.ravel()[0]+cm.ravel()[3])/sum(cm.ravel())
print('Accuracy is: %.4f' %acc)




# EXERCISE: What is the True Positive Rate ?
TPR_logit = cm.ravel()[3] / (cm.ravel()[3] + cm.ravel()[2] )
print('TPR is: %.4f' % TPR_logit)



# EXERCISE: What is the False Positive rate ?
FPR_logit = cm.ravel()[1] / (cm.ravel()[1] + cm.ravel()[0] )
print('FPR is: %.4f' % FPR_logit)

Accuracy is: 0.8459
TPR is: 0.0449
FPR is: 0.0062


### Take away: After running logistic regression, one can change the threshold probability. This will affect the predicted label. A smaller threshold will result in more observations being predicted as positive.